In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

def read_dat_file(filename):
    """
    Parse a .dat file of the form:
      # num_qubits f_mean f_min f_max w_mean w_min w_max b_mean b_min b_max r_mean r_min r_max
      <nq> <f_mean> <f_min> <f_max> <w_mean> <w_min> <w_max> <b_mean> <b_min> <b_max> <r_mean> <r_min> <r_max>
      ...
    Returns:
      - fraction (float) parsed from filename, if present
      - A list of (nq, f_mean, f_err_low, f_err_high, w_mean, w_err_low, w_err_high, b_mean, b_err_low, b_err_high, r_mean, r_err_low, r_err_high)
        where f_err_low = f_mean - f_min, f_err_high = f_max - f_mean, etc.
    """
    # Attempt to parse fraction from the filename fraction_{f}_cost.dat or fraction_{f}_time.dat
    # e.g. fraction_0.1_cost.dat => frac = 0.1
    base = os.path.basename(filename)
    # base might be "fraction_0.3_cost.dat"
    # let's extract the second piece after 'fraction_'
    fraction = None
    try:
        frac_part = base.split('_')[1]  # e.g. "0.3"
        fraction = float(frac_part)
    except:
        fraction = None

    rows = []
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            parts = line.split()
            # Expect 1 + 12 = 13 columns
            # 0: nq
            # 1..3 => f_mean, f_min, f_max
            # 4..6 => w_mean, w_min, w_max
            # 7..9 => b_mean, b_min, b_max
            # 10..12 => r_mean, r_min, r_max
            nq = int(parts[0])
            fmean, fmin_, fmax_ = map(float, parts[1:4])
            wmean, wmin_, wmax_ = map(float, parts[4:7])
            bmean, bmin_, bmax_ = map(float, parts[7:10])
            rmean, rmin_, rmax_ = map(float, parts[10:13])

            # Convert min/max => error bar
            f_err_low  = fmean - fmin_
            f_err_high = fmax_ - fmean
            w_err_low  = wmean - wmin_
            w_err_high = wmax_ - wmean
            b_err_low  = bmean - bmin_
            b_err_high = bmax_ - bmean
            r_err_low  = rmean - rmin_
            r_err_high = rmax_ - rmean

            rows.append((
                nq,
                fmean, (f_err_low, f_err_high),
                wmean, (w_err_low, w_err_high),
                bmean, (b_err_low, b_err_high),
                rmean, (r_err_low, r_err_high)
            ))

    # Sort rows by nq
    rows.sort(key=lambda x: x[0])
    return fraction, rows


def plot_barchart_per_fraction(fraction, rows, is_cost=True):
    """
    Create a bar chart for the given fraction using the data in 'rows'.

    Each 'rows' entry is:
      (nq,
       fmean, (f_err_low, f_err_high),
       wmean, (w_err_low, w_err_high),
       bmean, (b_err_low, b_err_high),
       rmean, (r_err_low, r_err_high)
      )

    We do a grouped bar chart with x-axis = list of 'nq'.
    For each 'nq', we have 4 bars: Fine, Window, Block, Recursive.
    Error bars come from the min/max differences.

    is_cost : bool => if True, we're plotting "Cost", else "Time".
    """
    # x-axis categories => num_qubits
    nq_list = [r[0] for r in rows]

    # We'll create an array for the index
    x_inds = np.arange(len(nq_list), dtype=float)

    # We'll define bar width
    bar_width = 0.2

    # Extract data
    f_mean  = [r[1] for r in rows]
    f_err   = [[r[2][0] for r in rows], [r[2][1] for r in rows]]  # [low, high] for each row

    w_mean  = [r[3] for r in rows]
    w_err   = [[r[4][0] for r in rows], [r[4][1] for r in rows]]

    b_mean  = [r[5] for r in rows]
    b_err   = [[r[6][0] for r in rows], [r[6][1] for r in rows]]

    r_mean  = [r[7] for r in rows]
    r_err   = [[r[8][0] for r in rows], [r[8][1] for r in rows]]

    # Initialize figure
    fig, ax = plt.subplots(figsize=(10, 6))

    # Because we have 4 bars per category => shift each bar group
    f_x = x_inds - bar_width*1.5
    w_x = x_inds - bar_width*0.5
    b_x = x_inds + bar_width*0.5
    r_x = x_inds + bar_width*1.5

    # Bar plots with error bars
    ax.bar(
        f_x, f_mean, 
        width=bar_width, 
        yerr=f_err, 
        capsize=3, 
        label='Fine',
        color='C0',
        error_kw=dict(ecolor='black')
    )
    ax.bar(
        w_x, w_mean, 
        width=bar_width, 
        yerr=w_err, 
        capsize=3, 
        label='Window',
        color='C1',
        error_kw=dict(ecolor='black')
    )
    ax.bar(
        b_x, b_mean, 
        width=bar_width, 
        yerr=b_err, 
        capsize=3, 
        label='Block',
        color='C2',
        error_kw=dict(ecolor='black')
    )
    ax.bar(
        r_x, r_mean, 
        width=bar_width, 
        yerr=r_err, 
        capsize=3, 
        label='Recursive',
        color='C3',
        error_kw=dict(ecolor='black')
    )

    # Labeling
    if is_cost:
        ax.set_ylabel("Cost")
        ax.set_title(f"Fraction = {fraction}, COST vs. #Qubits")
        outname = f"fraction_{fraction}_cost.png"
    else:
        ax.set_ylabel("Time (s)")
        ax.set_title(f"Fraction = {fraction}, TIME vs. #Qubits")
        outname = f"fraction_{fraction}_time.png"

    ax.set_xlabel("# Qubits")

    # Create x-ticks for the category
    ax.set_xticks(x_inds)
    ax.set_xticklabels([str(nq) for nq in nq_list], rotation=45)
    ax.legend()
    ax.grid(axis="y", alpha=0.3)

    plt.tight_layout()
    plt.savefig(outname, dpi=200)
    print(f"Saved figure: {outname}")
    plt.close(fig)


def main():
    data_dir = "dat_files"
    # 1) Collect all cost/time files
    cost_files = sorted(glob.glob(os.path.join(data_dir, "fraction_*_cost.dat")))
    time_files = sorted(glob.glob(os.path.join(data_dir, "fraction_*_time.dat")))

    # We'll do them in parallel or separately. Each cost file has a matching fraction.
    # But let's just loop over cost_files, time_files independently.

    # Plot cost
    for cfile in cost_files:
        fraction, rows = read_dat_file(cfile)
        if not rows:
            print(f"No data rows in {cfile}, skipping.")
            continue
        # fraction could be None if we couldn't parse from filename
        if fraction is None:
            fraction = 999  # or some fallback
        plot_barchart_per_fraction(fraction, rows, is_cost=True)

    # Plot time
    for tfile in time_files:
        fraction, rows = read_dat_file(tfile)
        if not rows:
            print(f"No data rows in {tfile}, skipping.")
            continue
        if fraction is None:
            fraction = 999
        plot_barchart_per_fraction(fraction, rows, is_cost=False)


if __name__ == "__main__":
    main()

In [1]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

def read_dat_file(filename):
    """
    Parse a .dat file of the form:
      # num_qubits f_mean f_min f_max w_mean w_min w_max b_mean b_min b_max r_mean r_min r_max
      <nq> <f_mean> <f_min> <f_max> <w_mean> <w_min> <w_max> <b_mean> <b_min> <b_max> <r_mean> <r_min> <r_max>
      ...
    Returns:
      - A list of (nq, f_mean, f_err_low, f_err_high, w_mean, w_err_low, w_err_high, b_mean, b_err_low, b_err_high, r_mean, r_err_low, r_err_high)
        where f_err_low = f_mean - f_min, f_err_high = f_max - f_mean, etc.
    """

    rows = []
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            parts = line.split()
            # Expect 1 + 12 = 13 columns
            # 0: nq
            # 1..3 => f_mean, f_min, f_max
            # 4..6 => w_mean, w_min, w_max
            # 7..9 => b_mean, b_min, b_max
            # 10..12 => r_mean, r_min, r_max
            nq = int(parts[0])
            fmean, fmin_, fmax_ = map(float, parts[1:4])
            wmean, wmin_, wmax_ = map(float, parts[4:7])
            bmean, bmin_, bmax_ = map(float, parts[7:10])
            rmean, rmin_, rmax_ = map(float, parts[10:13])

            # Convert min/max => error bar
            f_err_low  = fmean - fmin_
            f_err_high = fmax_ - fmean
            w_err_low  = wmean - wmin_
            w_err_high = wmax_ - wmean
            b_err_low  = bmean - bmin_
            b_err_high = bmax_ - bmean
            r_err_low  = rmean - rmin_
            r_err_high = rmax_ - rmean

            rows.append((
                nq,
                fmean, (f_err_low, f_err_high),
                wmean, (w_err_low, w_err_high),
                bmean, (b_err_low, b_err_high),
                rmean, (r_err_low, r_err_high)
            ))

    # Sort rows by nq
    rows.sort(key=lambda x: x[0])
    return rows


def plot_barchart(rows, is_cost=True):
    """
    Create a bar chart for the given using the data in 'rows'.

    Each 'rows' entry is:
      (nq,
       fmean, (f_err_low, f_err_high),
       wmean, (w_err_low, w_err_high),
       bmean, (b_err_low, b_err_high),
       rmean, (r_err_low, r_err_high)
      )

    We do a grouped bar chart with x-axis = list of 'nq'.
    For each 'nq', we have 4 bars: Fine, Window, Block, Recursive.
    Error bars come from the min/max differences.

    is_cost : bool => if True, we're plotting "Cost", else "Time".
    """
    # x-axis categories => num_qubits
    nq_list = [r[0] for r in rows]

    # We'll create an array for the index
    x_inds = np.arange(len(nq_list), dtype=float)

    # We'll define bar width
    bar_width = 0.2

    # Extract data
    f_mean  = [r[1] for r in rows]
    f_err   = [[r[2][0] for r in rows], [r[2][1] for r in rows]]  # [low, high] for each row

    w_mean  = [r[3] for r in rows]
    w_err   = [[r[4][0] for r in rows], [r[4][1] for r in rows]]

    b_mean  = [r[5] for r in rows]
    b_err   = [[r[6][0] for r in rows], [r[6][1] for r in rows]]

    r_mean  = [r[7] for r in rows]
    r_err   = [[r[8][0] for r in rows], [r[8][1] for r in rows]]

    # Initialize figure
    fig, ax = plt.subplots(figsize=(10, 6))

    # Because we have 4 bars per category => shift each bar group
    f_x = x_inds - bar_width*1.5
    w_x = x_inds - bar_width*0.5
    b_x = x_inds + bar_width*0.5
    r_x = x_inds + bar_width*1.5

    # Bar plots with error bars
    ax.bar(
        f_x, f_mean, 
        width=bar_width, 
        yerr=f_err, 
        capsize=3, 
        label='Fine',
        color='C0',
        error_kw=dict(ecolor='black')
    )
    ax.bar(
        w_x, w_mean, 
        width=bar_width, 
        yerr=w_err, 
        capsize=3, 
        label='Window',
        color='C1',
        error_kw=dict(ecolor='black')
    )
    ax.bar(
        b_x, b_mean, 
        width=bar_width, 
        yerr=b_err, 
        capsize=3, 
        label='Block',
        color='C2',
        error_kw=dict(ecolor='black')
    )
    ax.bar(
        r_x, r_mean, 
        width=bar_width, 
        yerr=r_err, 
        capsize=3, 
        label='Recursive',
        color='C3',
        error_kw=dict(ecolor='black')
    )

    # Labeling
    if is_cost:
        ax.set_ylabel("Cost")
        ax.set_title(f"COST vs. #Qubits")
        outname = f"_cost.png"
    else:
        ax.set_ylabel("Time (s)")
        ax.set_title(f"TIME vs. #Qubits")
        outname = f"_time.png"

    ax.set_xlabel("# Qubits")

    # Create x-ticks for the category
    ax.set_xticks(x_inds)
    ax.set_xticklabels([str(nq) for nq in nq_list], rotation=45)
    ax.legend()
    ax.grid(axis="y", alpha=0.3)

    plt.tight_layout()
    plt.savefig(outname, dpi=200)
    print(f"Saved figure: {outname}")
    plt.close(fig)


def main():
    data_dir = "dat_files"
    # 1) Collect all cost/time files
    cost_files = sorted(glob.glob(os.path.join(data_dir, input())))
    time_files = sorted(glob.glob(os.path.join(data_dir, input())))

    # We'll do them in parallel or separately. Each cost file has a matching fraction.
    # But let's just loop over cost_files, time_files independently.

    # Plot cost
    for cfile in cost_files:
        rows = read_dat_file(cfile)
        if not rows:
            print(f"No data rows in {cfile}, skipping.")
            continue
        plot_barchart(rows, is_cost=True)

    # Plot time
    for tfile in time_files:
        rows = read_dat_file(tfile)
        if not rows:
            print(f"No data rows in {tfile}, skipping.")
            continue

        plot_barchart(rows, is_cost=False)


if __name__ == "__main__":
    main()

Saved figure: _cost.png
Saved figure: _time.png
